In [1]:
import pyspark.sql.functions as f
from pyspark.sql.functions import desc
from pyspark.sql.functions import min, max, avg


In [2]:
df = spark.sql("select * from test_de_csv")
df.show(5)
df.printSchema()

+--------------------+--------------------+-----------+---------+--------------------+-------------------+------+
 userid| offerid|countrycode| category| merchant| utcdate|rating|
+--------------------+--------------------+-----------+---------+--------------------+-------------------+------+
d581f697c19367969...|be83df9772ec47fd2...| de| 125801|4740b6c83b6e12e42...|2016-06-15 09:43:32| 0|
b6208e4f5c0ea1a5d...|b75d6ee742849a622...| de|100020213|26467f203f42913b6...|2016-06-15 09:43:36| 0|
b4dfe41dcb584aee3...|3ba463bbdb2e24eac...| de|100020213|10698b6475abd54c5...|2016-06-15 09:44:02| 0|
d581f697c19367969...|be83df9772ec47fd2...| de| 125801|4740b6c83b6e12e42...|2016-06-15 09:44:04| 0|
351163eb97a830c6c...|b75d6ee742849a622...| de|100020213|26467f203f42913b6...|2016-06-15 09:44:12| 0|
+--------------------+--------------------+-----------+---------+--------------------+-------------------+------+
only showing top 5 rows

root
-- userid: string (nullable = true)
-- offerid: string (nullable = true)
-- countrycode: string (nullable = true)
-- category: integer (nullable = true)
-- merchant: string (nullable = true)
-- utcdate: timestamp (nullable = true)
-- rating: integer (nullable = true)

In [3]:
# extract out important features and removing unneeded columns
## 'hour' is expected to be an important feature
## 'day' is expected to be an important feature
## 'week' number may not be significant (in such a short time span) but captured anyways
## 'month' month my not be significant (in such a short time span) but captured anyways
# didn't observe any German holidays during or shortly after that can have an impact on click

df = df \
  .withColumn('day', f.dayofweek('utcdate')) \
  .withColumn('week', f.weekofyear('utcdate')) \
  .withColumn('month', f.month('utcdate')) \
  .withColumn('hour', f.hour('utcdate'))
df = df.drop(*['countrycode', 'utcdate'])

df.show(10)

df.createOrReplaceTempView('impressions')

# verifying our schema

df.printSchema()

# number of partitions

print("Number of partitions for 'df': ", df.rdd.getNumPartitions())


+--------------------+--------------------+---------+--------------------+------+---+----+-----+----+
 userid| offerid| category| merchant|rating|day|week|month|hour|
+--------------------+--------------------+---------+--------------------+------+---+----+-----+----+
d581f697c19367969...|be83df9772ec47fd2...| 125801|4740b6c83b6e12e42...| 0| 4| 24| 6| 9|
b6208e4f5c0ea1a5d...|b75d6ee742849a622...|100020213|26467f203f42913b6...| 0| 4| 24| 6| 9|
b4dfe41dcb584aee3...|3ba463bbdb2e24eac...|100020213|10698b6475abd54c5...| 0| 4| 24| 6| 9|
d581f697c19367969...|be83df9772ec47fd2...| 125801|4740b6c83b6e12e42...| 0| 4| 24| 6| 9|
351163eb97a830c6c...|b75d6ee742849a622...|100020213|26467f203f42913b6...| 0| 4| 24| 6| 9|
d581f697c19367969...|be83df9772ec47fd2...| 125801|4740b6c83b6e12e42...| 0| 4| 24| 6| 9|
c1054eb52d2a47aec...|3735290a415dc236b...| 125801|8bf8f87492a799528...| 0| 4| 24| 6| 9|
8163ddf01acd57116...|c5f63750c2b5b0166...|100020213|f3c93baa0cf443084...| 0| 4| 24| 6| 9|
b4380e45e42d8a18d...|146d635197d4d9092...|100010713|66863da8db7e6c51b...| 0| 4| 24| 6| 9|
badefb50c4de15ff6...|c5f63750c2b5b0166...|100020213|f3c93baa0cf443084...| 0| 4| 24| 6| 9|
+--------------------+--------------------+---------+--------------------+------+---+----+-----+----+
only showing top 10 rows

root
-- userid: string (nullable = true)
-- offerid: string (nullable = true)
-- category: integer (nullable = true)
-- merchant: string (nullable = true)
-- rating: integer (nullable = true)
-- day: integer (nullable = true)
-- week: integer (nullable = true)
-- month: integer (nullable = true)
-- hour: integer (nullable = true)

Number of partitions for 'df': 8

In [4]:
# looking at time frame discriptive stats

df.select(['hour', 'day', 'week', 'month', 'rating']).describe().show()

print("Number of distinct 'category': ", df.select('category').distinct().count())
print("Number of distinct 'merchant': ", df.select('merchant').distinct().count())
print("Number of distinct 'userid': ", df.select('userid').distinct().count())
print("Number of distinct 'offerid': ", df.select('offerid').distinct().count())

+-------+-----------------+------------------+------------------+-------------------+-------------------+
summary| hour| day| week| month| rating|
+-------+-----------------+------------------+------------------+-------------------+-------------------+
 count| 1919561| 1919561| 1919561| 1919561| 1919561|
 mean| 9.93217407521824|3.9443570691423715|24.504959206818643| 6.0025089069844615| 0.319952843384503|
 stddev|6.951414914815199| 1.548769034081369|0.7113425117138505|0.05002612991175306|0.46645807394142264|
 min| 0| 1| 24| 6| 0|
 max| 23| 7| 26| 7| 1|
+-------+-----------------+------------------+------------------+-------------------+-------------------+

Number of distinct 'category': 267
Number of distinct 'merchant': 738
Number of distinct 'userid': 278293
Number of distinct 'offerid': 380803

In [5]:
# Total number of clicks
df1 = df.groupby().sum('rating')
df1.show()

+-----------+
sum(rating)|
+-----------+
 614169|
+-----------+

In [6]:
# offer id that has received the max percentage of clicks
# we are assuming that the event with the most clickthrough rate and highest ad impression is most important

spark.sql('''
select offerid, sum(rating) as clicks, count(offerid) as impressions,
    sum(rating) * 100 / count(offerid) as pct
from test_de_csv
group by offerid
order by clicks desc, impressions desc, pct desc
limit 1
''' ).show(truncate=False)


+--------------------------------+------+-----------+-----------------+
offerid |clicks|impressions|pct |
+--------------------------------+------+-----------+-----------------+
0f2fcf95319f5c1e5745371351f521e5|41940 |49598 |84.55986128472922|
+--------------------------------+------+-----------+-----------------+

In [7]:
# user that has the max percentage of click
# we are assuming that the event with the most clickthrough rate and highest ad impression is most important

sample_df = df.sample(False, .5, )

spark.sql('''
select userid, sum(rating) as clicks, count(offerid) as impressions,
    sum(rating) * 100 / count(offerid) as pct
from test_de_csv
group by userid
order by clicks desc, impressions desc, pct desc
limit 1
''' ).show(truncate=False)

+----------------------------------------------------------------+------+-----------+------------------+
userid |clicks|impressions|pct |
+----------------------------------------------------------------+------+-----------+------------------+
241565de8ae842718ea3359c64246c38dc5b5335841d404d2b044a44a7cd7d23|4049 |15850 |25.545741324921135|
+----------------------------------------------------------------+------+-----------+------------------+

In [8]:
# Predict if the offer will be clicked or not

# 'userid', 'offerid' has too many categories
# 'month', 'week' has no predictive power since single or few categories
# unless we are able to group it make it smaller, we will drop it

df = df.drop(*['userid', 'offerid', 'month', 'week'])

# imports

from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler

# handling categorical columns

categorical_cols = ['category', 'merchant']

indexers = [StringIndexer(inputCol=col, outputCol=col+'idx') \
            .setHandleInvalid('keep') \
            for col in categorical_cols]

indexers_pipeline = Pipeline(stages=indexers)

df_indexed = indexers_pipeline.fit(df).transform(df).drop(*categorical_cols)

# creating assembler

feature_cols = df_indexed.columns[1:]

assembler = VectorAssembler(inputCols=feature_cols, outputCol='features')

df_assembled = assembler.transform(df_indexed)

df_ml = df_assembled.select(['rating', 'features'])

# Train-Test-Split

(df_train, df_test) = df_ml.randomSplit([0.6, 0.4], seed=42)

In [9]:
# random forest classifier (training set)

from pyspark.ml.classification import RandomForestClassifier as RFC

rfc = RFC(featuresCol='features', labelCol='rating', predictionCol='rating_pred', maxBins=1000, seed=42)

rfc_model = rfc.fit(df_train)

# gradient boosted tree classifier (training set)

from pyspark.ml.classification import GBTClassifier as GBTC

gbtc = GBTC(featuresCol='features', labelCol='rating', predictionCol='rating_pred', maxBins=1000, seed=42)

gbtc_model = gbtc.fit(df_train)


In [10]:
# model evaluation 

from pyspark.ml.evaluation import BinaryClassificationEvaluator as Evaluator

evaluator = Evaluator(labelCol='rating', rawPredictionCol='rating_pred')

# random forest classifier

rfc_train_pred = rfc_model.transform(df_train)
rfc_train_evaluation = evaluator.evaluate(rfc_train_pred)
print(f'Random Forest Classifier Accuracy (Train): {rfc_train_evaluation}')

rfc_test_pred = rfc_model.transform(df_test)
rfc_test_evaluation = evaluator.evaluate(rfc_test_pred)
print(f'Random Forest Classifier Accuracy (Test): {rfc_test_evaluation}\n')

# gradient boosted tree classifier 

gbtc_train_pred = gbtc_model.transform(df_train)
gbtc_train_evaluation = evaluator.evaluate(gbtc_train_pred)
print(f'Gradient Boosted Tree Classifier Accuracy (Train): {gbtc_train_evaluation}')

gbtc_test_pred = gbtc_model.transform(df_test)
gbtc_test_evaluation = evaluator.evaluate(gbtc_test_pred)
print(f'Gradient Boosted Tree Classifier Accuracy (Test): {gbtc_test_evaluation}\n')

Random Forest Classifier Accuracy (Train): 0.7922570130347584
Random Forest Classifier Accuracy (Test): 0.7926920781404597

Gradient Boosted Tree Classifier Accuracy (Train): 0.8335720671141592
Gradient Boosted Tree Classifier Accuracy (Test): 0.833173947483193

In [11]:
# confusion 'matrix' for  the random forest model (Train)
print('Confusion Matrix [Random Forest] (Train)')
rfc_train_pred.groupby('rating', 'rating_pred').count().show()

# confusion 'matrix' for  the random tree model (Train)
print('Confusion Matrix [Random Forest] (Test)')
rfc_test_pred.groupby('rating', 'rating_pred').count().show()

# confusion 'matrix' for  the gradient boosted tree model (Train)
print('Confusion Matrix [Gradient Boosted Tree] (Train)')
gbtc_train_pred.groupby('rating', 'rating_pred').count().show()

# confusion 'matrix' for  the gradient boosted tree model (Train)
print('Confusion Matrix [Gradient Boosted Tree] (Test)')
gbtc_test_pred.groupby('rating', 'rating_pred').count().show()

Confusion Matrix [Random Forest] (Train)
+------+-----------+------+
rating|rating_pred| count|
+------+-----------+------+
 1| 0.0| 93019|
 0| 0.0|655977|
 1| 1.0|275272|
 0| 1.0|127669|
+------+-----------+------+

Confusion Matrix [Random Forest] (Test)
+------+-----------+------+
rating|rating_pred| count|
+------+-----------+------+
 1| 0.0| 62057|
 0| 0.0|437105|
 1| 1.0|183821|
 0| 1.0| 84641|
+------+-----------+------+

Confusion Matrix [Gradient Boosted Tree] (Train)
+------+-----------+------+
rating|rating_pred| count|
+------+-----------+------+
 1| 0.0| 80250|
 0| 0.0|693560|
 1| 1.0|288041|
 0| 1.0| 90086|
+------+-----------+------+

Confusion Matrix [Gradient Boosted Tree] (Test)
+------+-----------+------+
rating|rating_pred| count|
+------+-----------+------+
 1| 0.0| 53723|
 0| 0.0|461663|
 1| 1.0|192155|
 0| 1.0| 60083|
+------+-----------+------+

In [12]:
rfc_pred = rfc_model.transform(df_ml)
# confusion 'matrix' for  the random forest model (Full)
print('\nConfusion Matrix [Random Forest]')
rfc_pred.groupby('rating', 'rating_pred').count().show()

# accuracy for entire dataset
rfc_pred =rfc_model.transform(df_ml)
rfc_evaluation = evaluator.evaluate(rfc_pred)
print(f'Random Forest Classifier Accuracy (Full): {rfc_evaluation}\n')


gbtc_pred = gbtc_model.transform(df_ml)
# confusion 'matrix' for  the gradient boosted tree model (Full)
print('\nConfusion Matrix [Gradient Boosted Tree]')
gbtc_pred.groupby('rating', 'rating_pred').count().show()

# accuracy for entire dataset
gbtc_pred = gbtc_model.transform(df_ml)
gbtc_evaluation = evaluator.evaluate(gbtc_pred)
print(f'Gradient Boosted Tree Classifier Accuracy (Full): {gbtc_evaluation}\n')


Confusion Matrix [Random Forest]
+------+-----------+-------+
rating|rating_pred| count|
+------+-----------+-------+
 1| 0.0| 155076|
 0| 0.0|1093082|
 1| 1.0| 459093|
 0| 1.0| 212310|
+------+-----------+-------+

Random Forest Classifier Accuracy (Full): 0.7924309612870397


Confusion Matrix [Gradient Boosted Tree]
+------+-----------+-------+
rating|rating_pred| count|
+------+-----------+-------+
 1| 0.0| 133973|
 0| 0.0|1155223|
 1| 1.0| 480196|
 0| 1.0| 150169|
+------+-----------+-------+

Gradient Boosted Tree Classifier Accuracy (Full): 0.8334127486181797

# Business Scenario

_In a situation, where a mobile advertisement company has this data as historical data. Each impression cost the advertisement company 1 cent, and each click costs the advertisement company $1. It has been given by merchants that for each impression the ROI for the merchant is 10 cents and for each click, the ROI for the merchant is $10._

_The advertisement company has $1000 to run the advertisement campaigns in the next 7 days. 
Based on the above historical dataset, identify the {offerid, merchantid} combination (or combinations) that the advertisement agency should run in this campaign._

In [14]:
# starting dataframe for reference

df.show(10)

+---------+--------------------+------+---+----+
 category| merchant|rating|day|hour|
+---------+--------------------+------+---+----+
 125801|4740b6c83b6e12e42...| 0| 4| 9|
100020213|26467f203f42913b6...| 0| 4| 9|
100020213|10698b6475abd54c5...| 0| 4| 9|
 125801|4740b6c83b6e12e42...| 0| 4| 9|
100020213|26467f203f42913b6...| 0| 4| 9|
 125801|4740b6c83b6e12e42...| 0| 4| 9|
 125801|8bf8f87492a799528...| 0| 4| 9|
100020213|f3c93baa0cf443084...| 0| 4| 9|
100010713|66863da8db7e6c51b...| 0| 4| 9|
100020213|f3c93baa0cf443084...| 0| 4| 9|
+---------+--------------------+------+---+----+
only showing top 10 rows

In [15]:
# Goal: We are trying to stay under advertising budget while attaining max ROI over a 7-day period
#
# Which merchant/offerid has best ROI for $1000 budget?


# calculate cost and roi

cost_vs_roi = spark.sql('''
select merchant, offerid, 
    concat(cast(week as string),'-',cast(day as string)) as day, 
    sum(rating) as clicks, count(offerid) as impressions,
    (count(offerid) * .01 + sum(rating)) as cost, 
    (count(offerid) + sum(rating) * 10) as roi
from impressions
group by offerid, merchant, week, day
order by day
''' )

cost_vs_roi.show(10)

cost_vs_roi.createOrReplaceTempView('cost_vs_roi')


+--------------------+--------------------+----+------+-----------+----+---+
 merchant| offerid| day|clicks|impressions|cost|roi|
+--------------------+--------------------+----+------+-----------+----+---+
9fae16c5686c07ae0...|bc5f87f70a215674b...|24-1| 0| 2|0.02| 2|
3235c2b68a5379a72...|caeba7cd03c67ba61...|24-1| 0| 4|0.04| 4|
fca91704667a53350...|cf1e71d28fac1fc0b...|24-1| 5| 8|5.08| 58|
26467f203f42913b6...|6b64825bb03b6391f...|24-1| 0| 4|0.04| 4|
154f65f908a740682...|8053e90d9436dd7c7...|24-1| 0| 1|0.01| 1|
a9d4a044a70b5a02f...|91181e1a8b668c2ba...|24-1| 1| 1|1.01| 11|
ac459db73b6bc490c...|e67b798338eca05bc...|24-1| 0| 1|0.01| 1|
ff1bd73f19ccc0984...|cc90276737ddde2a9...|24-1| 0| 5|0.05| 5|
c1d50531fe802776b...|94cf9dba8a622b5a8...|24-1| 1| 1|1.01| 11|
81dc9575e12600c50...|f9b3698accdede806...|24-1| 1| 12|1.12| 22|
+--------------------+--------------------+----+------+-----------+----+---+
only showing top 10 rows

In [16]:
# merchant/offerid combination

group_by_mod = spark.sql('''
select merchant, offerid, day, sum(cost) as total_cost, sum(roi) as total_roi
from cost_vs_roi
group by merchant, offerid, day
order by merchant, day, total_cost desc, total_roi desc
''')

group_by_mod.show(10)

group_by_mod.createOrReplaceTempView('group_by_mod')

+--------------------+--------------------+----+----------+---------+
 merchant| offerid| day|total_cost|total_roi|
+--------------------+--------------------+----+----------+---------+
002d1cfd4f3092ab0...|c9d54b65f6b1ba1e9...|24-1| 1.01| 11|
002d1cfd4f3092ab0...|40984497df60e975e...|24-1| 1.01| 11|
002d1cfd4f3092ab0...|b117430dd82c673a8...|24-1| 1.01| 11|
002d1cfd4f3092ab0...|299a41bdda149a021...|24-1| 1.01| 11|
002d1cfd4f3092ab0...|2001c998e0751a96b...|24-1| 1.01| 11|
002d1cfd4f3092ab0...|cc75f91e3c3a91b16...|24-1| 1.01| 11|
002d1cfd4f3092ab0...|9bbab1a79387abeb9...|24-1| 1.01| 11|
002d1cfd4f3092ab0...|f311e8808fa42fcdb...|24-1| 1.01| 11|
002d1cfd4f3092ab0...|4597ca4fccb51dafb...|24-1| 1.01| 11|
002d1cfd4f3092ab0...|1d41feab66bb54ac0...|24-1| 1.01| 11|
+--------------------+--------------------+----+----------+---------+
only showing top 10 rows

In [17]:
# now we want to look at the cost/roi in a 7 day window

seven_days = spark.sql('''
select merchant, offerid, day,
    sum(total_cost) over (partition by merchant, offerid 
                    order by day 
                    rows between 3 preceding and 3 following) as 7day_cost,
    sum(total_roi) over (partition by merchant, offerid
                    order by day 
                    rows between 3 preceding and 3 following) as 7day_roi
from group_by_mod
order by merchant, day
''')

seven_days.show(10)

seven_days.createOrReplaceTempView('seven_days')

+--------------------+--------------------+----+---------+--------+
 merchant| offerid| day|7day_cost|7day_roi|
+--------------------+--------------------+----+---------+--------+
002d1cfd4f3092ab0...|f5fc3796355200259...|24-1| 3.03| 33|
002d1cfd4f3092ab0...|eaece87327294557d...|24-1| 0.01| 1|
002d1cfd4f3092ab0...|f311e8808fa42fcdb...|24-1| 2.02| 22|
002d1cfd4f3092ab0...|fc2c189f526eb11c4...|24-1| 0.01| 1|
002d1cfd4f3092ab0...|40984497df60e975e...|24-1| 1.01| 11|
002d1cfd4f3092ab0...|0965bea9dbc78105d...|24-1| 0.01| 1|
002d1cfd4f3092ab0...|e1f67c60bdb28c38c...|24-1| 0.01| 1|
002d1cfd4f3092ab0...|a2c9328a05a8d71e8...|24-1| 0.01| 1|
002d1cfd4f3092ab0...|b559d632c643c5f0d...|24-1| 3.03| 33|
002d1cfd4f3092ab0...|7a5442d67f6960ea3...|24-1| 0.01| 1|
+--------------------+--------------------+----+---------+--------+
only showing top 10 rows

In [18]:
%sql

/* find the max roi while staying under budget */

select merchant, offerid, 7day_cost, 7day_roi
from seven_days
where 7day_cost < 1000
order by 7day_roi desc
limit 1

merchant,offerid,7day_cost,7day_roi
21a509189fb0875c3732590121ff3fc86da770b0628c1812e82f023ee2e0f683,19754ec121b3a99fff3967646942de67,789.58,78058


The goal here is to find offers for a user based on other users behavior. Here we will be using the _alternate least square (ALS)_ technique. ALS uses a matrix factorization technique in order to rank a user for a specific item.

In [20]:
# Prepping the data for analysis
# more clicks is evidence that a user likes the offer

users_offers_df = spark.sql('''
select userid, offerid, sum(rating) as rating
from test_de_csv
group by userid, offerid
''')

users_offers_df.show(10)

users_offers_df.createOrReplaceTempView('users_offers')

+--------------------+--------------------+------+
 userid| offerid|rating|
+--------------------+--------------------+------+
31491e43ee80f462c...|b75d6ee742849a622...| 0|
309f4bea413d687fe...|3735290a415dc236b...| 0|
6d20233754ce173be...|be83df9772ec47fd2...| 0|
308551c78f1894702...|5ac4398e4d8ad4167...| 0|
378cf197bec7e7fba...|499948cb3e3dc5fc6...| 1|
e7e7aac5d7c12f87f...|0d57d64e3d80f0e93...| 1|
adadd06750db0fb15...|19754ec121b3a99ff...| 0|
f961fca9f700f154e...|b75d6ee742849a622...| 0|
0b531a668c3ea03d3...|b75d6ee742849a622...| 0|
15c1271ec48c6b078...|5ac4398e4d8ad4167...| 0|
+--------------------+--------------------+------+
only showing top 10 rows

In [21]:
# checking sparsity of our data

sparsity = spark.sql('''
select 
  count(distinct(userid)) as ndu, 
  count(distinct(offerid)) as ndo, 
  count(rating) as nr,
  (1.0 - (count(rating) * 1.0) / (count(distinct(userid)) * count(distinct(offerid)))) as sparsity
from users_offers
''')
sparsity.show()

+------+------+------+-----------------+
 ndu| ndo| nr| sparsity|
+------+------+------+-----------------+
278293|380803|955929|0.999990979658218|
+------+------+------+-----------------+

In [22]:
# df = spark.sql("select * from users_offers")

# # offer with fewest clicks
# print(df.groupBy("offerid").count().select(min("count")).show())

# # offer with most clicks
# print(df.groupBy("offerid").count().select(max("count")).show())

# # average offer clicks
# print(df.groupBy("offerid").count().select(avg("count")).show())

# # average number of ratings per user
# print(df.groupBy("userid").count().select(avg("count")).show())

In [23]:
users_offers_df.sort(desc('rating')).show()

+--------------------+--------------------+------+
 userid| offerid|rating|
+--------------------+--------------------+------+
241565de8ae842718...|0f2fcf95319f5c1e5...| 1585|
7ac3d127d10f10e40...|0f2fcf95319f5c1e5...| 1255|
6cd69ee2f32ceec59...|0f2fcf95319f5c1e5...| 869|
241565de8ae842718...|d1b4365db02dd6936...| 804|
241565de8ae842718...|deafd09a713ed5c1b...| 804|
a66ae9d19b3125c83...|eb0389774fca117ee...| 695|
a66ae9d19b3125c83...|0f2fcf95319f5c1e5...| 682|
a3f43fc1e723ee926...|0f2fcf95319f5c1e5...| 637|
3fd3ffd7624b0525a...|0f2fcf95319f5c1e5...| 600|
68460e53b6de94c80...|0f2fcf95319f5c1e5...| 549|
ac47f1435465a5ea7...|0f2fcf95319f5c1e5...| 535|
95f9fdc63f091ea93...|0f2fcf95319f5c1e5...| 529|
7ac3d127d10f10e40...|10cddfdb5717d7a59...| 519|
7ac3d127d10f10e40...|146d635197d4d9092...| 519|
955e9014a41099b48...|0f2fcf95319f5c1e5...| 472|
a66ae9d19b3125c83...|d1b4365db02dd6936...| 472|
291e99c3da3b1fc01...|0f2fcf95319f5c1e5...| 461|
88985b5cf4e2f21b6...|0f2fcf95319f5c1e5...| 448|
6956b4fc49238790a...|0f2fcf95319f5c1e5...| 439|
2279b03c00974bddb...|0f2fcf95319f5c1e5...| 438|
+--------------------+--------------------+------+
only showing top 20 rows

In [24]:
rating_max = users_offers_df.agg({'rating': 'max'}).collect()[0][0]
rating_min = users_offers_df.agg({'rating': 'min'}).collect()[0][0]

print(f'Max: {rating_max}')
print(f'Min: {rating_min}')

Max: 1585
Min: 0

In [25]:
# # normalizing function
# def normRatings(rating):
#   return (rating - rating_min) / (rating_max - rating_min)

# from pyspark.sql.types import FloatType
# # defining a UDF method
# udfNormRatings = f.udf(normRatings, FloatType())

# # create a normalized column
# users_offers_df = users_offers_df.withColumn('rating_norm', udfNormRatings(users_offers_df.rating))

# users_offers_df.show()

+--------------------+--------------------+------+-----------+
 userid| offerid|rating|rating_norm|
+--------------------+--------------------+------+-----------+
31491e43ee80f462c...|b75d6ee742849a622...| 0| 0.0|
309f4bea413d687fe...|3735290a415dc236b...| 0| 0.0|
6d20233754ce173be...|be83df9772ec47fd2...| 0| 0.0|
308551c78f1894702...|5ac4398e4d8ad4167...| 0| 0.0|
378cf197bec7e7fba...|499948cb3e3dc5fc6...| 1|6.309148E-4|
e7e7aac5d7c12f87f...|0d57d64e3d80f0e93...| 1|6.309148E-4|
adadd06750db0fb15...|19754ec121b3a99ff...| 0| 0.0|
f961fca9f700f154e...|b75d6ee742849a622...| 0| 0.0|
0b531a668c3ea03d3...|b75d6ee742849a622...| 0| 0.0|
15c1271ec48c6b078...|5ac4398e4d8ad4167...| 0| 0.0|
f6b3df8f694d7fe43...|f2365684e053de7a6...| 0| 0.0|
7f2fdece5364d0d73...|d862cf209f246fbed...| 0| 0.0|
4bb89c3db1865849c...|9cf33663f2cdd9b65...| 0| 0.0|
4bb89c3db1865849c...|0d3cf2a16e6c0071e...| 0| 0.0|
0d9513cc2418b8e91...|5d785d7e0c41cacd1...| 1|6.309148E-4|
4bb89c3db1865849c...|fd0aeede5e5c81def...| 0| 0.0|
4bb89c3db1865849c...|01aee5bc0f0e12845...| 0| 0.0|
4bb89c3db1865849c...|256cd6125d0bf7964...| 0| 0.0|
fbe033a6c0c71e97f...|52fc02e61810cd68a...| 1|6.309148E-4|
2d1c075d215366751...|f7a59b0b052bd8754...| 0| 0.0|
+--------------------+--------------------+------+-----------+
only showing top 20 rows

In [26]:
# userid and offerid must be integers
# we will use monotonically_increasing_id to generate unique ids

from pyspark.sql.functions import monotonically_increasing_id

# use one partition
unique_users_df = users_offers_df.select('userid').distinct().coalesce(1)
# persist id
unique_users_df = unique_users_df.withColumn('userIntId', monotonically_increasing_id()).persist()
unique_users_df.show(10)

# use one partition
unique_offers_df = users_offers_df.select('offerid').distinct().coalesce(1)
# pesist id
unique_offers_df = unique_offers_df.withColumn('offerIntId', monotonically_increasing_id()).persist()
unique_offers_df.show(10)

+--------------------+---------+
 userid|userIntId|
+--------------------+---------+
b77b0c1ac4ea361ac...| 0|
e6d516dafd4f4eb69...| 1|
593cb5e4835536d4c...| 2|
0d8d6706e19e220ff...| 3|
b56b27a6c0933e86a...| 4|
862654278ca9ebb9c...| 5|
6ab0a34b167e718d7...| 6|
8bc59563ae8a28f61...| 7|
a1eb9ebee4f697c9e...| 8|
c0077ff44e80b7725...| 9|
+--------------------+---------+
only showing top 10 rows

+--------------------+----------+
 offerid|offerIntId|
+--------------------+----------+
0036cbdb333d6a14e...| 0|
b264a61974a59fb6e...| 1|
c1c8ddce56dbd3f9d...| 2|
064ff6b7b2cf41877...| 3|
b241578f932341b4e...| 4|
539f2b2a0e1fd50dc...| 5|
496ddf765abbb2619...| 6|
9af86fd71581da00f...| 7|
7a28f447c26c76547...| 8|
eb1e1595c2687f661...| 9|
+--------------------+----------+
only showing top 10 rows

In [27]:
# join the dataframe

ratings_df = users_offers_df\
  .join(unique_users_df, 'userid', 'left')\
  .join(unique_offers_df, 'offerid', 'left')
ratings_df.show(10)

+--------------------+--------------------+------+---------+----------+
 offerid| userid|rating|userIntId|offerIntId|
+--------------------+--------------------+------+---------+----------+
0006dba399cc98517...|4bb89c3db1865849c...| 0| 111395| 508|
002a364289ec376e2...|69064805abc8a7b54...| 1| 11135| 247|
002a364289ec376e2...|b34e805edcff0284b...| 1| 50335| 247|
002a364289ec376e2...|eefa0a80edcb55908...| 1| 150652| 247|
002a364289ec376e2...|90b274b6a99e1c6db...| 1| 122793| 247|
002a364289ec376e2...|96169893687a5bf55...| 1| 6013| 247|
002a364289ec376e2...|cfe61facf627f3d9c...| 1| 115833| 247|
002a364289ec376e2...|e523e1ce2a056c68b...| 1| 189551| 247|
002a364289ec376e2...|b4fa76c3ea553202e...| 1| 143310| 247|
002a364289ec376e2...|85c2d726da416fa21...| 1| 273366| 247|
+--------------------+--------------------+------+---------+----------+
only showing top 10 rows

In [28]:
# train, test, split
(X_train, X_test) = ratings_df.randomSplit([.7,.3], seed = 42)

# ALS
from pyspark.ml.recommendation import ALS
als_param = ALS(userCol='userIntId', 
                itemCol='offerIntId', 
                ratingCol='rating', 
                rank=20,
                maxIter=5,
                regParam=.1,
                coldStartStrategy='drop',
                nonnegative=True,
                implicitPrefs=False)

In [29]:
# build the model with the training data
als_model = als_param.fit(X_train)

# making predictions based on the test set
pred_test = als_model.transform(X_test)

In [30]:
pred_test.sort('userIntId').sort('offerIntId').sort(desc('prediction')).show()

+--------------------+--------------------+------+---------+----------+----------+
 offerid| userid|rating|userIntId|offerIntId|prediction|
+--------------------+--------------------+------+---------+----------+----------+
0f2fcf95319f5c1e5...|241565de8ae842718...| 1585| 223969| 323666| 633.33966|
0f2fcf95319f5c1e5...|4332c2e5cff098120...| 332| 26458| 323666| 451.5022|
deafd09a713ed5c1b...|4332c2e5cff098120...| 206| 26458| 192215| 431.41785|
0f2fcf95319f5c1e5...|7ac3d127d10f10e40...| 1255| 243513| 323666| 414.93628|
0f2fcf95319f5c1e5...|f0fbac7eb4c2c0ded...| 353| 167020| 323666| 412.20544|
d1b4365db02dd6936...|955e9014a41099b48...| 259| 18174| 72422| 379.91846|
146d635197d4d9092...|7ac3d127d10f10e40...| 519| 243513| 354131| 373.19452|
0f2fcf95319f5c1e5...|2279b03c00974bddb...| 438| 277016| 323666| 369.9884|
0f2fcf95319f5c1e5...|b4380e45e42d8a18d...| 342| 223848| 323666| 353.52283|
deafd09a713ed5c1b...|6956b4fc49238790a...| 130| 100345| 192215| 335.02884|
deafd09a713ed5c1b...|ac47f1435465a5ea7...| 338| 97633| 192215| 323.43146|
0f2fcf95319f5c1e5...|ce57395c3fe3f1037...| 319| 121381| 323666| 319.04614|
0f2fcf95319f5c1e5...|dd3194182ac3cc7dd...| 358| 105860| 323666| 307.75385|
0f2fcf95319f5c1e5...|6268507f1d7ab3511...| 282| 76867| 323666| 303.02667|
deafd09a713ed5c1b...|5936b296c93d34846...| 188| 201851| 192215| 301.56128|
0f2fcf95319f5c1e5...|548d1933657c4bc49...| 394| 112937| 323666| 298.0905|
0f2fcf95319f5c1e5...|fd82bceb96006341c...| 376| 36543| 323666| 297.77002|
0f2fcf95319f5c1e5...|1fe333a0a86706a7d...| 340| 157245| 323666| 292.58032|
eb0389774fca117ee...|241565de8ae842718...| 0| 223969| 245594| 292.0196|
deafd09a713ed5c1b...|58e9aa1b861081bdb...| 222| 230834| 192215| 291.60767|
+--------------------+--------------------+------+---------+----------+----------+
only showing top 20 rows

In [31]:
# model evaluation
from pyspark.ml.evaluation import RegressionEvaluator

eval = RegressionEvaluator(metricName='rmse',
                          labelCol='rating',
                          predictionCol='prediction')

rmse = eval.evaluate(pred_test)
print(rmse)

5.599242072990451

In [32]:
# *** THE BELOW WILL NOT RUN ON THE COMMUNITY VERSION. HOWEVER, WE COULD DO A GRID SEARCH TO TUNE IN THE HYPERPARAMETERS LIKE BELOW. ***

# hyperparameter tuning using Grid Search and CrossValidator

from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

param_grid = ParamGridBuilder() \
            .addGrid(als_param.rank, [25, 50, 100]) \
            .addGrid(als_param.maxIter, [10, 25, 50]) \
            .addGrid(als_param.regParam, [.01, .05, .1, .15]) \
            .build()

cv = CrossValidator(estimator=als_param,
                   estimatorParamMaps=param_grid,
                   evaluator=eval,
                   numFolds=5)

In [33]:
# fit cross-validator
cv_model = cv.fit(X_train)

# extract the tuned model
tuned_model = cv_model.bestModel

print(tuned_model.getRank(), tuned_model.getMaxIter(), tuned_model.getRegParam())

java.net.ConnectException: Connection refused
	at sun.nio.ch.SocketChannelImpl.checkConnect(Native Method)
	at sun.nio.ch.SocketChannelImpl.finishConnect(SocketChannelImpl.java:714)
	at sun.nio.ch.SocketAdaptor.connect(SocketAdaptor.java:122)
	at org.eclipse.jetty8.client.SelectConnector.startConnection(SelectConnector.java:82)
	at org.eclipse.jetty8.client.HttpDestination.startNewConnection(HttpDestination.java:292)
	at org.eclipse.jetty8.client.HttpDestination.doSend(HttpDestination.java:597)
	at org.eclipse.jetty8.client.HttpDestination.send(HttpDestination.java:532)
	at org.eclipse.jetty8.client.HttpClient.send(HttpClient.java:180)
	at com.databricks.rpc.JettyClient.$anonfun$sendRawExchange$2(JettyClient.scala:286)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:238)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:233)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:230)
	at com.databricks.rpc.AbstractJettyClient.withAttributionContext(BaseJettyClient.scala:85)
	at com.databricks.rpc.JettyClient.sendRawExchange(JettyClient.scala:270)
	at com.databricks.rpc.JettyClient.sendExchange(JettyClient.scala:186)
	at com.databricks.rpc.JettyClient.sendAsync(JettyClient.scala:142)
	at com.databricks.rpc.DynamicJettyClient.sendAsync(BaseJettyClient.scala:571)
	at com.databricks.rpc.BoundRPCClient.sendAsync(BoundRPCClient.scala:54)
	at com.databricks.backend.daemon.driver.DriverClient.send(DriverClient.scala:107)
	at com.databricks.backend.daemon.driver.DriverClient.$anonfun$poll$1(DriverClient.scala:455)
	at com.databricks.util.FutureUtils$.retry(FutureUtils.scala:87)
	at com.databricks.util.FutureUtils$$anonfun$retry$2.applyOrElse(FutureUtils.scala:91)
	at com.databricks.util.FutureUtils$$anonfun$retry$2.applyOrElse(FutureUtils.scala:89)
	at scala.concurrent.Future.$anonfun$recoverWith$1(Future.scala:417)
	at scala.concurrent.impl.Promise.$anonfun$transformWith$1(Promise.scala:41)
	at scala.concurrent.impl.CallbackRunnable.run(Promise.scala:64)
	at com.databricks.threading.NamedExecutor$$anon$2.$anonfun$run$1(NamedExecutor.scala:326)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:238)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:233)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:230)
	at com.databricks.threading.NamedExecutor.withAttributionContext(NamedExecutor.scala:265)
	at com.databricks.threading.NamedExecutor$$anon$2.run(NamedExecutor.scala:326)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1149)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:748)

In [34]:
# TODO: evaluator